In [1]:
import gmaps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

# Google developer API key
from api_keys import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [2]:
#introducing Vehicle data
vehicle_df = pd.read_csv("Data/vehicle_data.csv.gz", compression="gzip")

#take care of NaN velue
vehicle_df = vehicle_df.dropna()

#creating dataframe for car's for selective years
vehicle_year_df = vehicle_df.loc[vehicle_df['model_year'].astype(int) <= 2009]
#vehicle_year_df.head()

In [3]:
#introducing zip code coordinates
zip_df = pd.read_csv('Data/us-zip-code-latitude-and-longitude.csv')
zip_df = zip_df.rename (columns = {"Zip" : "zip"})

#take care of NaN velue
zip_df = zip_df.dropna()
#zip_df.head()

In [4]:
#add zip code coordinates to vehicle age mean dataframe
vehicle_2009_df = pd.merge(vehicle_year_df, zip_df, on=['zip','zip'], how="left")

#take care of NaN velue
vehicle_2009_df = vehicle_2009_df.dropna()
vehicle_2009_df.head()

,vin,registration_class,zip,model_year,body_type,fuel_type,City,State,Latitude,Longitude,Timezone,Daylight savings time flag
0,999407G3573,PAS,12866,1976,SUBN,GAS,Saratoga Springs,NY,43.080399,-73.77038,-5.0,1.0
1,9992313,PAS,13316,1924,CONV,GAS,Camden,NY,43.377083,-75.74261,-5.0,1.0
2,998867,PAS,11501,1952,2DSD,GAS,Mineola,NY,40.747188,-73.63808,-5.0,1.0
3,9983692,PAS,10940,1936,2DSD,GAS,Middletown,NY,41.449364,-74.43951,-5.0,1.0
4,99782010525,PAS,11225,1978,2DSD,GAS,Brooklyn,NY,40.662892,-73.95509,-5.0,1.0


In [5]:
# find mean of vehicle age by zip code
veh_number_df = vehicle_2009_df.groupby('zip').agg({'vin':'count'})
veh_number_df = veh_number_df.reset_index()
veh_number_df

,zip,vin
0,10001,952
1,10002,3516
2,10003,1584
3,10005,149
4,10006,75
...,...,...
2045,14901,2581
2046,14902,57
2047,14903,1586
2048,14904,2972


In [6]:
#add zip code coordinates to vehicle age mean dataframe
vehicle_09_df = pd.merge(veh_number_df, zip_df, on=['zip','zip'], how="left")

#take care of NaN velue
vehicle_09_df = vehicle_09_df.dropna()
vehicle_09_df.head()

,zip,vin,City,State,Latitude,Longitude,Timezone,Daylight savings time flag
0,10001,952,New York,NY,40.750742,-73.99653,-5,1
1,10002,3516,New York,NY,40.717040,-73.98700,-5,1
2,10003,1584,New York,NY,40.732509,-73.98935,-5,1
3,10005,149,New York,NY,40.706019,-74.00858,-5,1
4,10006,75,New York,NY,40.707904,-74.01342,-5,1


In [7]:
#Plot Heatmap for vehicle age by zip code
locations = vehicle_09_df[["Latitude", "Longitude"]].astype(float)
vehicle_number = vehicle_09_df["vin"].astype(float)


#create heat layer for vehicle age by zip
heat_layer = gmaps.heatmap_layer(locations, weights=vehicle_number, 
                                  dissipating=False, max_intensity=500,                                  
                                  point_radius=0.05)


#combine both maps
fig = gmaps.figure(map_type="HYBRID")
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))